In [1]:
import os, glob, random, time, sys, pickle
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa, librosa.display
import soundfile as sf

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# import torch.optim as optim
import torch_optimizer as optim
from torchvision import transforms, models, datasets
# from torchlibrosa.stft import Spectrogram, LogmelFilterBank
# from torchlibrosa.augmentation import SpecAugmentation
from torchaudio.transforms import AmplitudeToDB
from transformers import get_linear_schedule_with_warmup

from tqdm import tqdm
# from tqdm.notebook import tqdm
from functools import partial
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, ConfusionMatrixDisplay, confusion_matrix
from skimage.transform import resize
from PIL import Image, ImageOps

import timm
from timm.models.efficientnet import tf_efficientnet_b0_ns, tf_efficientnet_b1_ns, tf_efficientnet_b2_ns, tf_efficientnet_b3_ns, tf_efficientnet_b4_ns

from os import walk
from pandas import DataFrame
from fastai.vision.widgets import *
import zipfile
from ipyfilechooser import FileChooser

import warnings
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/home/rasieira/anaconda3/envs/murcielagos/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
class args:
    arquitectura = 'resnet34',
    num_clases = 7,
    spec = {
        'n_fft' : 3072, #2048 3072 #16384, #384000 // 2 // 512 * 16, # 512 * 2
        'hop_length' : 768, #512, 768, 1024
        'fmax' : 96000, #384000 // 2,  192000 // 2
        'fmin' : 8000,
        'tramo': 1.5
    }
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# def test_epoch(model, loader):
#     model.eval()
#     y_pred = []
#     with torch.no_grad():
#         t = tqdm(loader)
#         for i, sample in enumerate(t):
#             input = sample['image'].float().to(device)
#             bs = input.size(0)
            
#             output = model(input)
#             y_pred.append(torch.sigmoid(output).detach().cpu().numpy())
#         t.close()
#     y_pred = np.vstack(y_pred)
#     return y_pred

In [4]:
def dataset_inferencia(path):      
    imgs=[]
    paths=[]
    for (dirpath, dirnames, filenames) in walk(path):
        for t in filenames:
            completo=path+'/'+t
            paths.append(str(completo))

    for a in paths:
        y, sr = sf.read(a)
        len_crop = int(args.spec['tramo'] * sr)

        # Repetimos la señal si es menor y luego cortamos
        while len(y)<len_crop:
            y = np.concatenate([y, y])
        y = y[0:len_crop]

        f_min = args.spec['fmin']
        f_max = args.spec['fmax']

        # Frecuencias y hop_length proporcionales a la sr
        if sr==192000:
            n_fft= args.spec['n_fft'] // 2
            hop_length = args.spec['hop_length'] // 2
        if sr==256000:
            n_fft= args.spec['n_fft'] * 2 // 3
            hop_length = args.spec['hop_length'] * 2 // 3
        if sr==384000:
            n_fft= args.spec['n_fft']
            hop_length = args.spec['hop_length']

        # Banda de frecuencias a filtrar
        frecuencias = np.arange(0, 1 + n_fft / 2) * sr / n_fft
        indices_freq = (frecuencias >= f_min) & (frecuencias <= f_max)

        # Reduce el paso a la mitad si sr==192000
        img = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length, win_length=n_fft, window='hamming'))**2
        img = librosa.power_to_db(img, ref=np.max)
        img = img[indices_freq,:]
        img = torch.from_numpy(img).unsqueeze(0)
        imgs.append(img)
    return {"images" : imgs,'paths' : paths}

In [5]:
def Model():
    model = timm.create_model(args.arquitectura[0], pretrained=True, num_classes=args.num_clases[0], in_chans=1)
    return model

In [6]:
out_pl = widgets.Output()
out_pl2 = widgets.Output()

In [7]:
def on_click_classify(change):
    data=dataset_inferencia(fc.selected_path)
    df = DataFrame (data['paths'],columns=['paths'])
    out_pl.clear_output()
    out_pl2.clear_output()
    model_path=fc1.selected_path+'/'+fc1.selected_filename
    model=Model()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    y_pred = []
#     with torch.no_grad():
#         for i in data['images']:
#             input = i.float().to(device)
#             bs = input.size(0)
#             output = model(input)
#             y_pred.append(torch.sigmoid(output).detach().cpu().numpy())
#     y_pred = np.vstack(y_pred)
    with out_pl: display(df)
    with out_pl2:display(model_path)

In [8]:
fc = FileChooser('../')

# Print the selected path, filename, or both
if fc.selected_path is not None:
    print(fc.selected_path)
if fc.selected_filename is not None:
    print(fc.selected_filename)
if fc.selected is not None:
    print(fc.selected)

# Change defaults and reset the dialog
fc.default_path = '../'
fc.reset()

# Shorthand reset
fc.reset(path='../')

# Change hidden files
fc.show_hidden = True

# Show or hide folder icons
fc.use_dir_icons = True

# Switch to folder-only mode
fc.show_only_dirs = True

# Change the title (use '' to hide)
fc.title = '<b>Selecciona el path</b>'

fc1 = FileChooser('../')

# Print the selected path, filename, or both
if fc1.selected_path is not None:
    print(fc1.selected_path)
if fc1.selected_filename is not None:
    print(fc1.selected_filename)
if fc1.selected is not None:
    print(fc1.selected)

# Change defaults and reset the dialog
fc1.default_path = '../'
fc1.reset()

# Shorthand reset
fc1.reset(path='../')

# Change hidden files
fc1.show_hidden = True

# Show or hide folder icons
fc1.use_dir_icons = True

# Switch to folder-only mode
fc1.show_only_dirs = False

# Change the title (use '' to hide)
fc1.title = '<b>Selecciona el Modelo (.bin)</b>'

In [9]:
btn_run = widgets.Button(description='Sacar Predicciones')
btn_run.on_click(on_click_classify)

In [10]:
VBox([fc,fc1, out_pl,out_pl2,btn_run])